In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

* This notebook contains the complete code to extract the online news related to an specific firm, in this case Microsoft (MSFT).

    1. All the news have been extracted from https://business.financialpost.com/

In [1]:
import requests
import urllib
import sys
import numpy  as np
import pandas as pd

from bs4 import BeautifulSoup

# 1. Extract web links according to a signature and the number of pages


In [2]:
def get_internal_links(ur):
    
    info  = BeautifulSoup(requests.get(ur,allow_redirects = True).content,'html.parser').find_all("h4",{"class":"entry-title"})
    links  = [ a['href'] for each_link in info for a in each_link.find_all('a', href=True)]

    return(links)   

In [3]:
def extract_link_of_news(k_word,n_of_page):
    
    news_df                  = pd.DataFrame()    
    
    link =  "https://business.financialpost.com/page/"+str(n_of_page)+"?s="+k_word
    

    news_df["internal_urls"]  = get_internal_links(link)
    news_df["principal_url"]  = link    
    news_df["n_of_page"]      = n_of_page
    
    return(news_df[["n_of_page","principal_url","internal_urls"]])

In [4]:
n_of_pages = 20
df = pd.concat([extract_link_of_news("microsoft",i) for i in range(1,n_of_pages+1)],ignore_index = True)
df

,n_of_page,principal_url,internal_urls
0,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/busines...
1,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/technology/...
2,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/press-r...
3,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...
4,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...
...,...,...,...
195,20,https://business.financialpost.com/page/20?s=m...,https://business.financialpost.com/technology/...
196,20,https://business.financialpost.com/page/20?s=m...,https://business.financialpost.com/technology/...
197,20,https://business.financialpost.com/page/20?s=m...,https://business.financialpost.com/technology/...
198,20,https://business.financialpost.com/page/20?s=m...,https://business.financialpost.com/pmn/busines...


# 2. Extract the date and text from each link 

In [5]:
def extract_date_and_text(ur):
    url = ur
    
    try:
        soup         = BeautifulSoup(requests.get(url).content, 'html.parser')
        g = 0
    
    except:   
        the_type, the_value, the_traceback = sys.exc_info()
        g = 1
    

    if(g==0):

        if(soup.find("div",class_ = "entry-content").find_all("p")):
            text_of_news = soup.find("div",class_ = "entry-content").find_all("p")
        elif(soup.find("div",class_ = "entry-content").find("p")):
            text_of_news = soup.find("div",class_ = "entry-content").find("p")
        else:
            text_of_news = "NO TEXT"

        final_date   = soup.find("time")["datetime"]

        get_text_vec = [i.text for i in text_of_news]
        final_text = "".join(get_text_vec)
        
        
    else: final_date = "";final_text = ""
    
    return(final_date,final_text)

In [8]:
a = np.vectorize(extract_date_and_text)
test = df[0:10].copy()
test["internals_dates"],test["internals_text"] = a(test["internal_urls"][0:10])
test.head()

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
0,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/busines...,2019-08-27T09:25:03-04:00,AMSTERDAM — Microsoft is remotely collecting d...
1,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/technology/...,2015-03-18T11:01:07-04:00,"ATLANTA, Georgia – This week, over 10,000 peop..."
2,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/press-r...,2018-09-25T09:02:40-04:00,"NEW YORK — Pexip, an award-winning video conf..."
3,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...,2012-11-13T12:02:39-05:00,Microsoft’s Windows boss Steven Sinofsky is ou...
4,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...,2013-04-15T11:56:02-04:00,Microsoft is working on a smartwatch with a 1....


In [9]:
a = np.vectorize(extract_date_and_text)
df["internals_dates"],df["internals_text"] = a(df["internal_urls"])

In [10]:
df.head()

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
0,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/busines...,2019-08-27T09:25:03-04:00,AMSTERDAM — Microsoft is remotely collecting d...
1,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/technology/...,2015-03-18T11:01:07-04:00,"ATLANTA, Georgia – This week, over 10,000 peop..."
2,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/pmn/press-r...,2018-09-25T09:02:40-04:00,"NEW YORK — Pexip, an award-winning video conf..."
3,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...,2012-11-13T12:02:39-05:00,Microsoft’s Windows boss Steven Sinofsky is ou...
4,1,https://business.financialpost.com/page/1?s=mi...,https://business.financialpost.com/business-in...,2013-04-15T11:56:02-04:00,Microsoft is working on a smartwatch with a 1....


In [11]:
df.loc[df["internals_text"] == "NO TEXT"]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text


In [12]:
df.loc[df["internals_text"] == ""]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text


In [14]:
df.to_pickle("data sets/microsoft_complete_text.pkl")